# Function Calling

Read:

https://platform.openai.com/docs/guides/function-calling

https://cookbook.openai.com/examples/how_to_call_functions_with_chat_models


In [1]:
from openai import OpenAI
import json
import requests
from dotenv import load_dotenv, find_dotenv
import os

_ : bool = load_dotenv(find_dotenv()) # read local .env file

client : OpenAI = OpenAI()


## The basic sequence of steps for function calling is as follows:

1. Call the model with the user query and a set of functions defined in the functions parameter.
2. The model can choose to call one or more functions; if so, the content will be a stringified JSON object adhering to your custom schema (note: the model may hallucinate parameters).
3. Parse the string into JSON in your code, and call your function with the provided arguments if they exist.
4. Call the model again by appending the function response as a new message, and let the model summarize the results back to the user.

https://www.linkedin.com/pulse/azure-openai-function-calling-tarun-sharma/

In [2]:
# Function that gets current weather from open weather api
def get_current_weather(city: str, unit:str="metric")->str:
    """Get the current weather of a given city"""
    open_weather_api = os.getenv('OPENWEATHER_API_KEY')
    res = requests.get(f"https://api.openweathermap.org/data/2.5/weather?q={city}&units={unit}&appid={open_weather_api}")
    res = res.json()

    return json.dumps(res)

### Types

https://github.com/openai/openai-python/tree/main/src/openai/types/chat


In [3]:
from openai.types.chat.chat_completion import ChatCompletionMessage, ChatCompletion

def run_conversation(main_request: str)->str:
    # Step 1: send the conversation and available functions to the model
    messages = [{"role": "user", "content": main_request}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "city": {
                            "type": "string",
                            "description": "The city, e.g. San Francisco",
                        },
                        "unit": {
                            "type": "string",
                            "enum": ["standard", "metric", "imperial"], 
                            "description": "The units of temperature; default should be celsius e.g standard equals kelvin, metric equals celsius and imperial equals fahrenheit"
                        },
                    },
                    "required": ["city"],
                },
            },
        }
    ]

    # First Request
    response: ChatCompletion = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message: ChatCompletionMessage = response.choices[0].message

    tool_calls = response_message.tool_calls

    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        
        messages.append(response_message)  # extend conversation with assistant's reply
        
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                city=function_args.get("city"),
                unit=function_args.get("unit"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        second_response: ChatCompletion = client.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response.choices[0].message.content

In [4]:
run_conversation("What's the weather of Peshawar and Chicago?")

'Currently in Peshawar, the weather is 16.08°C with light rain and haze, while in Chicago, the weather is 40.78°F with a clear sky.'